In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

/software/python_ucs4-2.7.12-el7-x86_64/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import re, os, h5py, sys, math
from datetime import datetime

path = '/project/ftchong/qoc/pranav/'
file_name = datetime.today().strftime('%h%d')

Populating the interactive namespace from numpy and matplotlib


In [4]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}

In [5]:
U = np.array([[ 0.15435964+0.02879184j, -0.36651249-0.06836352j,
          0.        +0.j        , -0.34298943-0.0639759j ,
          0.03619322+0.00675092j, -0.0859374 -0.01602942j,
          0.        +0.j        , -0.08042187-0.01500064j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.24155837-0.04505653j,  0.57355769+0.1069825j ,
          0.        +0.j        ,  0.53674631+0.10011628j],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.52516669-0.0979564j ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.12313758-0.02296816j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.82183665+0.15329258j,  0.        +0.j        ],
        [-0.48399397-0.09027668j, -0.2038379 -0.03802074j,
          0.        +0.j        ,  0.        +0.j        ,
         -0.11348368-0.02116747j, -0.04779455-0.00891485j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.7574052 +0.14127455j,  0.3189872 +0.05949889j,
          0.        +0.j        ,  0.        +0.j        ],
        [-0.31258963-0.05830559j,  0.55890096+0.10424866j,
          0.        +0.j        ,  0.52355698+0.09765615j,
         -0.00551769-0.00102918j,  0.01060584+0.00197825j,
          0.        +0.j        ,  0.00434069+0.00080964j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.18969458-0.03538267j,  0.33289765+0.06209353j,
          0.        +0.j        ,  0.36771901+0.06858857j],
        [ 0.63850575+0.1190969j ,  0.2249654 +0.04196153j,
          0.        +0.j        ,  0.20552438+0.03833531j,
          0.14911778+0.0278141j , -0.33036586-0.06162129j,
          0.        +0.j        , -0.25612389-0.04777335j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.32700627+0.06099465j,  0.33965596+0.06335412j,
          0.        +0.j        , -0.21578385-0.04024895j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.22441165+0.04185825j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.95708819-0.17852029j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.13804212+0.02574822j,  0.20408945+0.03806766j,
          0.        +0.j        , -0.15596166-0.02909065j,
         -0.8236366 -0.15362832j, -0.31265925-0.05831858j,
          0.        +0.j        , -0.03656885-0.00682098j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.08170125+0.01523928j, -0.19399195-0.03618423j,
          0.        +0.j        ,  0.24406538+0.04552415j],
        [ 0.05688748+0.0106109j , -0.135074  -0.0251946j ,
          0.        +0.j        ,  0.16993946+0.03169785j,
         -0.24261814-0.0452542j ,  0.57607404+0.10745186j,
          0.        +0.j        , -0.72477096-0.13518746j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.28894168-0.05389467j,  0.68606492+0.12796784j,
          0.        +0.j        ,  0.64203273+0.11975477j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.98304547+0.18336196j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.90597536+0.1689865j ,  0.3815587 +0.07117001j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.24919821+0.04648155j, -0.59169779-0.11036607j,
          0.        +0.j        ,  0.74442754+0.1388539j ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.35046147+0.06536961j,  0.05474569+0.01021141j,
          0.        +0.j        ,  0.04868376+0.00908071j,
         -0.24573177-0.04583497j,  0.59553998+0.11108273j,
          0.        +0.j        ,  0.5843357 +0.10899286j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.13448462+0.02508466j,  0.24922051+0.0464857j ,
          0.        +0.j        , -0.03861082-0.00720186j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.80013608+0.1492449j ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.18761057+0.03499395j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.53940978+0.10061308j,  0.        +0.j        ],
        [ 0.12901096+0.02406369j,  0.36425303+0.06794208j,
          0.        +0.j        , -0.33117406-0.06177204j,
          0.36060589+0.06726179j,  0.22453978+0.04188215j,
          0.        +0.j        , -0.07765148-0.0144839j ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.38469417+0.07175485j, -0.32297726-0.06024314j,
          0.        +0.j        ,  0.51825636+0.09666745j],
        [ 0.20283139+0.037833j  , -0.48160413-0.08983092j,
          0.        +0.j        ,  0.60591636+0.11301818j,
          0.04755855+0.00887083j, -0.11292332-0.02106295j,
          0.        +0.j        ,  0.14207123+0.02649975j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.13673829+0.02550503j, -0.32467225-0.06055929j,
          0.        +0.j        ,  0.40847704+0.07619093j]])

In [6]:
N = 4
connected_qubit_pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = transmon_gate(U, d)

In [11]:
total_time = 34.8
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path = path)

data saved at: /project/ftchong/qoc/pranav/00004_Feb10.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.95e-01; Runtime: 1.1s; Iterations = 0, grads =   4.784e-03, unitary_metric = 0.99987
Error = :2.80e-01; Runtime: 56.8s; Iterations = 100, grads =   1.068e+00, unitary_metric = 0.99979
Error = :1.40e-01; Runtime: 111.9s; Iterations = 200, grads =   2.458e-02, unitary_metric = 0.99964
Error = :7.20e-02; Runtime: 166.9s; Iterations = 300, grads =   1.901e-01, unitary_metric = 0.99959
Error = :3.43e-02; Runtime: 222.3s; Iterations = 400, grads =   6.418e-03, unitary_metric = 0.99956
Error = :1.88e-02; Runtime: 277.5s; Iterations = 500, grads =   6.164e-02, unitary_metric = 0.99964
Error = :1.39e-02; 

----
## Unitary Slices

In [5]:
unitaries = [matrix([[0.        +0.j        , 0.        +0.j        ,
         0.        -0.70710678j, 0.70710678+0.j        ],
        [0.70710678+0.j        , 0.        -0.70710678j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        -0.70710678j, 0.70710678+0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.70710678+0.j        , 0.        -0.70710678j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.54084856-0.84111999j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.54084856+0.84111999j]]), matrix([[ 0.70710678+0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
         -0.70710678+0.j],
        [ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.70710678+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.56897351+0.82235585j]]), matrix([[ 0.5+0.j ,  0. +0.5j,  0. +0.5j,  0.5+0.j ],
        [ 0. -0.5j,  0.5+0.j , -0.5+0.j ,  0. +0.5j],
        [ 0.5+0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.j ],
        [ 0. -0.5j, -0.5+0.j ,  0.5+0.j ,  0. +0.5j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.56897351+0.82235585j]]), matrix([[ 0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.35355339j,  0.35355339+0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.35355339j, -0.35355339+0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        -0.35355339j,  0.35355339+0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        -0.35355339j, -0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        -0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.35355339+0.j        ,  0.        -0.35355339j,
          0.        -0.35355339j,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.35355339+0.j        ,  0.        +0.35355339j,
          0.        -0.35355339j, -0.35355339+0.j        ,
          0.        +0.35355339j, -0.35355339+0.j        ,
          0.35355339+0.j        ,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ]]), matrix([[ 1.       +0.j        ,  0.       +0.j        ],
        [ 0.       +0.j        , -0.1038772-0.99459013j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]]), matrix([[ 1.       +0.j        ,  0.       +0.j        ],
        [ 0.       +0.j        , -0.1038772-0.99459013j]]), matrix([[ 0.70710678+0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
         -0.70710678+0.j],
        [ 0.70710678+0.j,  0.        +0.j, -0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.70710678+0.j]]), matrix([[ 1.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        , -0.80425367+0.59428616j]]), matrix([[ 0.5+0.j ,  0. +0.5j,  0. +0.5j,  0.5+0.j ],
        [ 0. -0.5j,  0.5+0.j , -0.5+0.j ,  0. +0.5j],
        [ 0.5+0.j ,  0. -0.5j,  0. -0.5j,  0.5+0.j ],
        [ 0. -0.5j, -0.5+0.j ,  0.5+0.j ,  0. +0.5j]]), matrix([[ 1.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        , -0.80425367+0.59428616j]]), matrix([[ 0.25-0.25j,  0.25-0.25j, -0.25+0.25j, -0.25+0.25j,  0.25+0.25j,
          0.25+0.25j,  0.25+0.25j,  0.25+0.25j],
        [ 0.25-0.25j, -0.25+0.25j,  0.25-0.25j, -0.25+0.25j,  0.25+0.25j,
         -0.25-0.25j, -0.25-0.25j,  0.25+0.25j],
        [ 0.25+0.25j,  0.25+0.25j, -0.25-0.25j, -0.25-0.25j,  0.25-0.25j,
          0.25-0.25j,  0.25-0.25j,  0.25-0.25j],
        [ 0.25+0.25j, -0.25-0.25j,  0.25+0.25j, -0.25-0.25j,  0.25-0.25j,
         -0.25+0.25j, -0.25+0.25j,  0.25-0.25j],
        [ 0.25+0.25j,  0.25+0.25j,  0.25+0.25j,  0.25+0.25j,  0.25-0.25j,
          0.25-0.25j, -0.25+0.25j, -0.25+0.25j],
        [ 0.25+0.25j, -0.25-0.25j, -0.25-0.25j,  0.25+0.25j,  0.25-0.25j,
         -0.25+0.25j,  0.25-0.25j, -0.25+0.25j],
        [ 0.25-0.25j,  0.25-0.25j,  0.25-0.25j,  0.25-0.25j,  0.25+0.25j,
          0.25+0.25j, -0.25-0.25j, -0.25-0.25j],
        [ 0.25-0.25j, -0.25+0.25j, -0.25+0.25j,  0.25-0.25j,  0.25+0.25j,
         -0.25-0.25j,  0.25+0.25j, -0.25-0.25j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778+0.34104278j]]), matrix([[ 0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ],
        [ 0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j, -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j],
        [ 0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778+0.34104278j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778+0.34104278j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. -0.5j,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778+0.34104278j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778-0.34104278j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778-0.34104278j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778-0.34104278j]]), matrix([[ 0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0.5+0.j ],
        [ 0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ],
        [ 0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. -0.5j, -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.94004778-0.34104278j]]), matrix([[ 0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
         -0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
         -0.70710678+0.j],
        [ 0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j, -0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
         -0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j, -0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j, -0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j, -0.70710678+0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j, -0.70710678+0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j],
        [ 0.        +0.j,  0.        +0.j,  0.70710678+0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.70710678+0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.70710678+0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434+0.30471263j]]), matrix([[ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434-0.30471263j]]), matrix([[ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434+0.30471263j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j, -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434+0.30471263j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434-0.30471263j]]), matrix([[ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j, -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434+0.30471263j]]), matrix([[ 0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j],
        [ 0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.5j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434-0.30471263j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.95244434-0.30471263j]]), matrix([[ 0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ],
        [-0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j],
        [-0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j],
        [ 0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j],
        [ 0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j, -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j],
        [ 0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j],
        [ 0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  -0.25j, -0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j],
        [ 0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ],
        [-0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j],
        [-0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j],
        [ 0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[ 0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j],
        [ 0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[ 0. -0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[ 0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0.5+0.j ,  0.5+0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,  0.5+0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.5j,  0. +0.5j,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.5j,  0. -0.5j,  0. -0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
         -0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0.5+0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j , -0.5+0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.5j,  0. +0.5j,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.5j,  0. -0.5j,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126+0.61310522j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
         0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126-0.61310522j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0.5+0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ]]), matrix([[1.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.79000126+0.61310522j]]), matrix([[ 0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        +0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        +0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        +0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        +0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        +0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        +0.35355339j,
          0.        +0.j        ,  0.        +0.j        ],
        [ 0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.35355339+0.j        ,  0.35355339+0.j        ],
        [ 0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ,
          0.        -0.35355339j,  0.        -0.35355339j,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.35355339+0.j        , -0.35355339+0.j        ]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008-0.9602533j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008+0.9602533j]]), matrix([[ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008-0.9602533j]]), matrix([[ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008-0.9602533j]]), matrix([[ 0.  +0.25j,  0.  +0.25j, -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
         -0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j],
        [ 0.  -0.25j,  0.  -0.25j, -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
         -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j],
        [ 0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j, -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  +0.25j, -0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  ,
          0.25+0.j  ],
        [-0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j, -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.  -0.25j,  0.  +0.25j, -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,
          0.  -0.25j],
        [ 0.  -0.25j,  0.  +0.25j,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
         -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,
          0.  +0.25j],
        [ 0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j, -0.25+0.j  , -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.25+0.j  ,
          0.25+0.j  ],
        [-0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j, -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.  -0.25j,  0.  +0.25j,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
         -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,
          0.  -0.25j],
        [ 0.  -0.25j,  0.  +0.25j, -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,
          0.  +0.25j],
        [ 0.  +0.25j,  0.  +0.25j,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
          0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j],
        [ 0.  -0.25j,  0.  -0.25j,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j],
        [ 0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  -0.25j, -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j, -0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.25+0.j  ,
         -0.25+0.j  ]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008+0.9602533j]]), matrix([[ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008+0.9602533j]]), matrix([[ 0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008-0.9602533j]]), matrix([[ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ],
        [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0. -0.5j,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. -0.5j],
        [ 0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j , -0.5+0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. -0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.5j,  0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ],
        [ 0. +0.j ,  0. +0.j ,  0. +0.j ,  0. -0.5j,  0. +0.j ,
         -0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,
          0. +0.j ],
        [ 0. -0.5j,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j , -0.5+0.j ,  0. +0.j ,  0.5+0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.5j,
          0. +0.j ],
        [ 0. +0.j ,  0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0. +0.j ,  0. +0.j ,  0. +0.5j,  0. +0.j ,  0. +0.5j,
          0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ,
          0.5+0.j ]]), matrix([[1.        +0.j       , 0.        +0.j       ],
        [0.        +0.j       , 0.27913008+0.9602533j]]), matrix([[ 0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  +0.25j,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,
          0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  +0.25j,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.25+0.j  ,  0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,
          0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  -0.25j,  0.  -0.25j, -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,
          0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j,  0.  +0.25j, -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j,  0.  -0.25j, -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  ],
        [ 0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,
          0.  -0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j,  0.  +0.25j, -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ],
        [ 0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  +0.25j],
        [ 0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,  0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  -0.25j],
        [ 0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,  0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  ,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  +0.25j],
        [ 0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  ,  0.  +0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  -0.25j],
        [ 0.  +0.25j,  0.  +0.25j,  0.  -0.25j,  0.  -0.25j, -0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j,
          0.  +0.25j],
        [ 0.  +0.25j,  0.  -0.25j,  0.  -0.25j,  0.  +0.25j, -0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
         -0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  +0.25j,
          0.  -0.25j],
        [ 0.  +0.25j,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j, -0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  , -0.25+0.j  ,
          0.25+0.j  , -0.25+0.j  ,  0.  -0.25j,  0.  +0.25j,  0.  -0.25j,
          0.  +0.25j],
        [ 0.  +0.25j,  0.  +0.25j,  0.  +0.25j,  0.  +0.25j, -0.25+0.j  ,
         -0.25+0.j  , -0.25+0.j  , -0.25+0.j  ,  0.25+0.j  ,  0.25+0.j  ,
          0.25+0.j  ,  0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,  0.  -0.25j,
          0.  -0.25j]])]

In [6]:
[unitary.shape for unitary in unitaries]

[(4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (16, 16),
 (2, 2),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (4, 4),
 (2, 2),
 (8, 8),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (8, 8),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (8, 8),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16),
 (2, 2),
 (16, 16)]

In [7]:
def binary_search_for_shortest_pulse_time(min_steps, max_steps):
    """Search between [min_steps, max_steps] (inclusive)."""
    while min_steps + 20 < max_steps:  # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('trying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        _, _, converged = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                         reg_coeffs=reg_coeffs,
                         use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                         show_plots=False, file_name=file_name, data_path=path, return_converged=True)
        if converged:
            max_steps = mid_steps
        else:
            min_steps = mid_steps

    return mid_steps / 20.0

In [20]:
max_iterations = 1000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}

In [ ]:
pulse_times = []
for U in unitaries:
    if U.shape == (2, 2):
        N = 1
        connected_qubit_pairs = []
    elif U.shape == (4, 4):
        N = 2
        connected_qubit_pairs = [(0, 1)]
    elif U.shape == (8, 8):
        N = 3
        connected_qubit_pairs = [(0, 1), (0, 2), (1, 2)]
    elif U.shape == (16, 16):
        N = 4
        connected_qubit_pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
    else:
        assert False, 'only working with <= 4 qubit blocks right now'
    
    H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
    Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
    states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
    maxA = hamiltonian.get_maxA(N, d)
    U = transmon_gate(U, d)
    pulse_time = binary_search_for_shortest_pulse_time(1, (2 ** N) * 3 * 20)
    print('***Pulse Time: %s***\n\n\n' % str(pulse_time))
    pulse_times.append(pulse_time)

trying total_time: 6.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00050_Feb10.h5
Using 4 Taylor terms and 5 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.97e-01; Runtime: 0.1s; Iterations = 0, grads =   3.656e-03, unitary_metric = 1.00003
Error = :1.06e-01; Runtime: 4.1s; Iterations = 100, grads =   2.440e-04, unitary_metric = 0.99997
Error = :5.37e-02; Runtime: 8.0s; Iterations = 200, grads =   8.715e-05, unitary_metric = 0.99995
Error = :3.83e-02; Runtime: 11.8s; Iterations = 300, grads =   5.892e-05, unitary_metric = 0.99996
Error = :3.16e-02; Runtime: 15.7s; Iterations = 400, grads =   6.147e-05, unitary_metric = 0.99997
Error = :2.48e-02; Runtime: 19.6s; Iterations = 500, grads =   4.395e-05, 

Error = :4.31e-02; Runtime: 34.0s; Iterations = 900, grads =   6.538e-05, unitary_metric = 0.99997
Error = :4.10e-02; Runtime: 37.8s; Iterations = 1000, grads =   5.743e-05, unitary_metric = 0.99995
data saved at: /project/ftchong/qoc/pranav/00057_Feb10.h5
trying total_time: 9.0 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00058_Feb10.h5
Using 4 Taylor terms and 5 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :1.00e+00; Runtime: 0.2s; Iterations = 0, grads =   1.009e-05, unitary_metric = 1.00001
Error = :4.45e-03; Runtime: 6.1s; Iterations = 100, grads =   9.374e-04, unitary_metric = 0.99996
Error = :9.34e-04; Runtime: 8.9s; Iterations = 148, grads =   1.648e-04, unitary_metric = 0.99996
data saved at: 

Error = :8.70e-01; Runtime: 0.3s; Iterations = 0, grads =   1.030e-02, unitary_metric = 1.00003
Error = :1.78e-03; Runtime: 6.2s; Iterations = 100, grads =   1.617e-04, unitary_metric = 0.99995
Error = :9.26e-04; Runtime: 6.7s; Iterations = 106, grads =   9.499e-05, unitary_metric = 0.99998
data saved at: /project/ftchong/qoc/pranav/00065_Feb10.h5
trying total_time: 7.5 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00066_Feb10.h5
Using 4 Taylor terms and 5 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.37e-01; Runtime: 0.2s; Iterations = 0, grads =   1.211e-01, unitary_metric = 1.00000
Error = :3.43e-02; Runtime: 5.1s; Iterations = 100, grads =   2.955e-04, unitary_metric = 0.99996
Error = :3.19e-03; R

Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.37e-01; Runtime: 0.2s; Iterations = 0, grads =   1.150e-02, unitary_metric = 1.00002
Error = :4.20e-03; Runtime: 5.1s; Iterations = 100, grads =   1.646e-04, unitary_metric = 0.99996
Error = :1.27e-03; Runtime: 9.8s; Iterations = 200, grads =   3.579e-06, unitary_metric = 0.99996
Error = :9.99e-04; Runtime: 11.5s; Iterations = 234, grads =   2.554e-06, unitary_metric = 0.99995
data saved at: /project/ftchong/qoc/pranav/00073_Feb10.h5
trying total_time: 6.75 for unitary of size (4, 4)
data saved at: /project/ftchong/qoc/pranav/00074_Feb10.h5
Using 4 Taylor terms and 5 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.49e-01; Runtime: 0.2s; Iteration

Initialized
Error = :1.00e+00; Runtime: 1.4s; Iterations = 0, grads =   5.335e-04, unitary_metric = 0.99986
Error = :2.54e-01; Runtime: 72.6s; Iterations = 100, grads =   4.472e-01, unitary_metric = 0.99959
Error = :1.04e-01; Runtime: 142.8s; Iterations = 200, grads =   3.254e-02, unitary_metric = 0.99949
Error = :4.08e-02; Runtime: 212.6s; Iterations = 300, grads =   2.048e-02, unitary_metric = 0.99944
Error = :9.47e-03; Runtime: 286.3s; Iterations = 400, grads =   1.650e-03, unitary_metric = 0.99942
Error = :3.13e-03; Runtime: 359.8s; Iterations = 500, grads =   2.536e-04, unitary_metric = 0.99943
Error = :1.84e-03; Runtime: 431.9s; Iterations = 600, grads =   8.227e-05, unitary_metric = 0.99941
Error = :1.32e-03; Runtime: 502.3s; Iterations = 700, grads =   2.052e-05, unitary_metric = 0.99938
Error = :1.03e-03; Runtime: 572.6s; Iterations = 800, grads =   3.498e-06, unitary_metric = 0.99938
Error = :9.79e-04; Runtime: 575.1s; Iterations = 802, grads =   3.414e-06, unitary_metric = 0

...

Error = :7.70e-02; Runtime: 13.8s; Iterations = 100, grads =   4.926e-03, unitary_metric = 0.99989
Error = :2.70e-02; Runtime: 26.6s; Iterations = 200, grads =   1.063e-03, unitary_metric = 0.99984
Error = :1.05e-02; Runtime: 39.6s; Iterations = 300, grads =   1.022e-04, unitary_metric = 0.99988
Error = :5.96e-03; Runtime: 52.5s; Iterations = 400, grads =   7.089e-06, unitary_metric = 0.99982
Error = :3.80e-03; Runtime: 65.4s; Iterations = 500, grads =   3.057e-06, unitary_metric = 0.99983
Error = :2.63e-03; Runtime: 78.3s; Iterations = 600, grads =   1.888e-06, unitary_metric = 0.99985
Error = :2.02e-03; Runtime: 91.4s; Iterations = 700, grads =   1.345e-06, unitary_metric = 0.99985
Error = :1.60e-03; Runtime: 104.2s; Iterations = 800, grads =   1.022e-06, unitary_metric = 0.99987
Error = :1.33e-03; Runtime: 117.0s; Iterations = 900, grads =   8.114e-07, unitary_metric = 0.99983
Error = :1.14e-03; Runtime: 129.8s; Iterations = 1000, grads =   6.646e-07, unitary_metric = 0.99982
data s

Error = :6.13e-02; Runtime: 181.4s; Iterations = 300, grads =   2.177e-02, unitary_metric = 0.99983
Error = :2.46e-02; Runtime: 241.1s; Iterations = 400, grads =   1.464e-03, unitary_metric = 0.99988
Error = :1.52e-02; Runtime: 300.9s; Iterations = 500, grads =   5.177e-04, unitary_metric = 0.99982
Error = :9.89e-03; Runtime: 360.7s; Iterations = 600, grads =   1.409e-04, unitary_metric = 0.99986
Error = :7.13e-03; Runtime: 420.5s; Iterations = 700, grads =   1.776e-05, unitary_metric = 0.99982
Error = :5.54e-03; Runtime: 480.1s; Iterations = 800, grads =   6.539e-06, unitary_metric = 0.99980
Error = :4.47e-03; Runtime: 539.6s; Iterations = 900, grads =   4.797e-06, unitary_metric = 0.99983
Error = :3.74e-03; Runtime: 599.2s; Iterations = 1000, grads =   3.832e-06, unitary_metric = 0.99982
data saved at: /project/ftchong/qoc/pranav/00117_Feb10.h5
trying total_time: 42.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00118_Feb10.h5
Using 6 Taylor terms and 6 Sca

Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.85e-01; Runtime: 0.1s; Iterations = 0, grads =   4.845e-02, unitary_metric = 1.00001
Error = :7.17e-03; Runtime: 1.0s; Iterations = 100, grads =   2.361e-04, unitary_metric = 1.00002
Error = :4.40e-03; Runtime: 1.9s; Iterations = 200, grads =   1.248e-04, unitary_metric = 1.00002
Error = :1.43e-03; Runtime: 2.7s; Iterations = 300, grads =   4.022e-05, unitary_metric = 1.00002
Error = :9.87e-04; Runtime: 3.0s; Iterations = 326, grads =   2.938e-05, unitary_metric = 1.00002
data saved at: /project/ftchong/qoc/pranav/00123_Feb10.h5
trying total_time: 0.75 for unitary of size (2, 2)
data saved at: /project/ftchong/qoc/pranav/00124_Feb10.h5
Using 6 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate prop

Error = :1.32e-01; Runtime: 193.2s; Iterations = 300, grads =   2.307e-02, unitary_metric = 0.99971
Error = :3.78e-02; Runtime: 256.9s; Iterations = 400, grads =   2.506e-02, unitary_metric = 0.99970
Error = :2.26e-02; Runtime: 320.9s; Iterations = 500, grads =   3.697e-03, unitary_metric = 0.99974
Error = :9.13e-03; Runtime: 384.8s; Iterations = 600, grads =   9.828e-04, unitary_metric = 0.99972
Error = :4.92e-03; Runtime: 448.9s; Iterations = 700, grads =   2.709e-04, unitary_metric = 0.99970
Error = :3.14e-03; Runtime: 512.6s; Iterations = 800, grads =   3.935e-05, unitary_metric = 0.99972
Error = :2.45e-03; Runtime: 576.2s; Iterations = 900, grads =   5.625e-06, unitary_metric = 0.99979
Error = :2.06e-03; Runtime: 640.0s; Iterations = 1000, grads =   2.578e-06, unitary_metric = 0.99979
data saved at: /project/ftchong/qoc/pranav/00129_Feb10.h5
trying total_time: 41.25 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00130_Feb10.h5
Using 6 Taylor terms and 6 Sc

Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.96e-01; Runtime: 1.2s; Iterations = 0, grads =   2.944e-03, unitary_metric = 0.99982
Error = :3.15e-01; Runtime: 68.3s; Iterations = 100, grads =   4.509e-01, unitary_metric = 1.00007
Error = :9.50e-02; Runtime: 133.7s; Iterations = 200, grads =   3.085e-02, unitary_metric = 0.99991
Error = :4.89e-02; Runtime: 199.6s; Iterations = 300, grads =   4.526e-02, unitary_metric = 0.99998
Error = :1.37e-02; Runtime: 265.2s; Iterations = 400, grads =   3.900e-03, unitary_metric = 0.99998
Error = :3.04e-03; Runtime: 330.9s; Iterations = 500, grads =   4.030e-04, unitary_metric = 0.99991
Error = :2.08e-03; Runtime: 396.7s; Iterations = 600, grads =   4.412e-04, unitary_metric = 0.99991
Error = :9.94e-04; Runtime: 451.6s; Iterations = 683, grads =   1.354e-04, unitary_metric = 0.99990


In [17]:
# Got through 24 of the slices before crashing. Resuming from here:
pulse_times = [6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 38.25, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 15.75, 0.75, 38.25, 0.75, 41.25, 0.75, 39.75, 0.75]

In [ ]:
for U in unitaries[24:]:
    if U.shape == (2, 2):
        N = 1
        connected_qubit_pairs = []
    elif U.shape == (4, 4):
        N = 2
        connected_qubit_pairs = [(0, 1)]
    elif U.shape == (8, 8):
        N = 3
        connected_qubit_pairs = [(0, 1), (0, 2), (1, 2)]
    elif U.shape == (16, 16):
        N = 4
        connected_qubit_pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
    else:
        assert False, 'only working with <= 4 qubit blocks right now'
    
    H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
    Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
    states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
    maxA = hamiltonian.get_maxA(N, d)
    U = transmon_gate(U, d)
    pulse_time = binary_search_for_shortest_pulse_time(1, (2 ** N) * 3 * 20)
    print('***Pulse Time: %s***\n\n\n' % str(pulse_time))
    pulse_times.append(pulse_time)
    print(pulse_times)

trying total_time: 24.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00002_Feb11.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.92e-01; Runtime: 0.9s; Iterations = 0, grads =   3.220e-03, unitary_metric = 0.99989
Error = :3.25e-01; Runtime: 41.6s; Iterations = 100, grads =   5.738e-02, unitary_metric = 0.99982
Error = :1.74e-01; Runtime: 81.8s; Iterations = 200, grads =   2.025e-02, unitary_metric = 0.99975
Error = :1.22e-01; Runtime: 121.9s; Iterations = 300, grads =   4.947e-04, unitary_metric = 0.99964
Error = :9.87e-02; Runtime: 162.4s; Iterations = 400, grads =   8.945e-05, unitary_metric = 0.99965
Error = :8.49e-02; Runtime: 202.4s; Iterations = 500, grads =   4.0

Error = :2.66e-01; Runtime: 64.7s; Iterations = 100, grads =   4.711e-01, unitary_metric = 0.99967
Error = :7.49e-02; Runtime: 127.1s; Iterations = 200, grads =   1.500e-02, unitary_metric = 0.99942
Error = :2.31e-02; Runtime: 189.9s; Iterations = 300, grads =   2.496e-03, unitary_metric = 0.99939
Error = :1.07e-02; Runtime: 252.5s; Iterations = 400, grads =   4.873e-04, unitary_metric = 0.99938
Error = :6.57e-03; Runtime: 315.3s; Iterations = 500, grads =   1.689e-04, unitary_metric = 0.99935
Error = :4.34e-03; Runtime: 377.8s; Iterations = 600, grads =   3.230e-05, unitary_metric = 0.99933
Error = :3.27e-03; Runtime: 440.2s; Iterations = 700, grads =   7.864e-06, unitary_metric = 0.99935
Error = :2.63e-03; Runtime: 502.9s; Iterations = 800, grads =   1.885e-06, unitary_metric = 0.99939
Error = :2.24e-03; Runtime: 565.4s; Iterations = 900, grads =   9.343e-07, unitary_metric = 0.99937
Error = :1.98e-03; Runtime: 627.9s; Iterations = 1000, grads =   7.034e-07, unitary_metric = 0.99937


Error = :1.85e-03; Runtime: 488.1s; Iterations = 700, grads =   2.626e-04, unitary_metric = 0.99972
Error = :9.95e-04; Runtime: 543.2s; Iterations = 779, grads =   6.838e-05, unitary_metric = 0.99966
data saved at: /project/ftchong/qoc/pranav/00013_Feb11.h5
trying total_time: 39.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00014_Feb11.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.91e-01; Runtime: 1.4s; Iterations = 0, grads =   2.739e-03, unitary_metric = 0.99987
Error = :3.17e-01; Runtime: 66.5s; Iterations = 100, grads =   6.297e-02, unitary_metric = 0.99981
Error = :7.60e-02; Runtime: 130.3s; Iterations = 200, grads =   3.538e-01, unitary_metric = 0.99954
Error = 

Error = :7.99e-02; Runtime: 51.0s; Iterations = 500, grads =   2.355e-05, unitary_metric = 0.99988
Error = :7.52e-02; Runtime: 61.1s; Iterations = 600, grads =   2.094e-05, unitary_metric = 0.99983
Error = :7.16e-02; Runtime: 71.2s; Iterations = 700, grads =   1.932e-05, unitary_metric = 0.99983
Error = :6.89e-02; Runtime: 81.2s; Iterations = 800, grads =   1.834e-05, unitary_metric = 0.99983
Error = :6.68e-02; Runtime: 91.3s; Iterations = 900, grads =   1.770e-05, unitary_metric = 0.99983
Error = :6.51e-02; Runtime: 101.4s; Iterations = 1000, grads =   1.718e-05, unitary_metric = 0.99984
data saved at: /project/ftchong/qoc/pranav/00020_Feb11.h5
trying total_time: 18.0 for unitary of size (8, 8)
data saved at: /project/ftchong/qoc/pranav/00021_Feb11.h5
Using 4 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Trainin

Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :7.83e-01; Runtime: 0.0s; Iterations = 0, grads =   2.684e-01, unitary_metric = 1.00001
Error = :9.14e-05; Runtime: 0.2s; Iterations = 17, grads =   3.158e-06, unitary_metric = 1.00001
data saved at: /project/ftchong/qoc/pranav/00027_Feb11.h5
***Pulse Time: 0.75***



[6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 38.25, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 15.75, 0.75, 38.25, 0.75, 41.25, 0.75, 39.75, 0.75, 38.25, 0.75, 38.25, 0.75, 15.75, 0.75]
trying total_time: 24.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00028_Feb11.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate prop

Error = :1.61e-03; Runtime: 552.5s; Iterations = 900, grads =   3.051e-06, unitary_metric = 0.99947
Error = :1.34e-03; Runtime: 613.5s; Iterations = 1000, grads =   1.279e-06, unitary_metric = 0.99950
data saved at: /project/ftchong/qoc/pranav/00032_Feb11.h5
trying total_time: 38.25 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00033_Feb11.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.99e-01; Runtime: 1.3s; Iterations = 0, grads =   6.353e-04, unitary_metric = 0.99983
Error = :3.34e-01; Runtime: 64.6s; Iterations = 100, grads =   3.883e-01, unitary_metric = 0.99968
Error = :9.52e-02; Runtime: 126.8s; Iterations = 200, grads =   1.823e-01, unitary_metric = 0.99956
Error 

Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.94e-01; Runtime: 1.3s; Iterations = 0, grads =   5.749e-03, unitary_metric = 0.99985
Error = :2.83e-01; Runtime: 68.2s; Iterations = 100, grads =   8.015e-02, unitary_metric = 0.99996
Error = :1.30e-01; Runtime: 133.8s; Iterations = 200, grads =   5.791e-01, unitary_metric = 0.99982
Error = :5.23e-02; Runtime: 199.2s; Iterations = 300, grads =   4.917e-02, unitary_metric = 0.99983
Error = :3.39e-02; Runtime: 264.9s; Iterations = 400, grads =   9.249e-03, unitary_metric = 0.99981
Error = :8.89e-03; Runtime: 330.3s; Iterations = 500, grads =   1.594e-03, unitary_metric = 0.99983
Error = :3.44e-03; Runtime: 396.1s; Iterations = 600, grads =   3.393e-04, unitary_metric = 0.99981
Error = :1.87e-03; Runtime: 461.6s; Iterations = 700, grads =   2.987e-05, unitary_metric = 0.99981
Error = :1.33e-03; Runtime: 527.6s; Iterations =

Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :4.86e-01; Runtime: 0.0s; Iterations = 0, grads =   4.010e-01, unitary_metric = 1.00001
Error = :1.84e-04; Runtime: 0.2s; Iterations = 12, grads =   3.365e-05, unitary_metric = 1.00001
data saved at: /project/ftchong/qoc/pranav/00045_Feb11.h5
***Pulse Time: 0.75***



[6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 38.25, 0.75, 6.75, 0.75, 6.75, 0.75, 6.75, 0.75, 15.75, 0.75, 38.25, 0.75, 41.25, 0.75, 39.75, 0.75, 38.25, 0.75, 38.25, 0.75, 15.75, 0.75, 38.25, 0.75, 41.25, 0.75]
trying total_time: 24.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/pranav/00046_Feb11.h5
Using 6 Taylor terms and 6 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training l